In [2]:
# -*- coding: utf-8 -*-
import math
import os
import re
import glob
import copy
import pandas as pd
from collections import defaultdict, deque
import matplotlib.pyplot as plt

###############################################
# 1) [필수] 지하철 노선 데이터 (분기·순환 고려)
###############################################
subway_lines = {
    #----------------------------------------------------
    # 1호선
    #----------------------------------------------------
    "1호선": [
        "연천","전곡","청산","소요산","동두천","보산","동두천중앙","지행","덕정","덕계","양주","녹양","가능","의정부","회룡","망월사","도봉산","도봉","방학","창동","녹천","월계","광운대","석계","신이문","외대앞","회기","청량리","제기동","신설동","동묘앞","동대문","종로5가","종로3가","종각","시청","서울역","남영","용산","노량진","대방","신길","영등포","신도림","구로",
        # 경인선 (구로 -> 인천)
        "구일","개봉","오류동","온수","역곡","소사","부천","중동","송내","부개","부평","백운","동암","간석","주안","도화","제물포","도원","동인천","인천",
        # 다시 구로 -> (경부선/장항선)
        "구로","가산디지털단지","독산","금천구청","석수","관악","안양","명학","금정","군포","당정","의왕","성균관대","화서","수원","세류","병점","세마","오산대","오산","진위","송탄","서정리","평택지제","평택","성환","직산","두정","천안","봉명","쌍용","아산","탕정","배방","온양온천","신창",

        "금천구청","광명",
        "병점","서동탄"
    ],
    #----------------------------------------------------
    # 2호선
    #----------------------------------------------------
    "2호선": [
        # 순환구간 (시청~시청)
        "시청","을지로입구","을지로3가","을지로4가","동대문역사문화공원","신당","상왕십리","왕십리","한양대","뚝섬","성수","건대입구","구의","강변","잠실나루","잠실","잠실새내","종합운동장","삼성","선릉","역삼","강남","교대","서초","방배","사당","낙성대","서울대입구","봉천","신림","신대방","구로디지털단지","대림","신도림","문래","영등포구청","당산","합정","홍대입구","신촌","이대","아현","충정로","시청",
        # 성수지선
        "성수","용답","신답","용두","신설동",
        # 신정지선
        "신도림","도림천","양천구청","신정네거리","까치산"
    ],
    #----------------------------------------------------
    # 3호선
    #----------------------------------------------------
    "3호선": [
        "대화","주엽","정발산","마두","백석","대곡","화정","원당","원흥","삼송","지축",
        "구파발","연신내","불광","녹번","홍제","무악재","독립문","경복궁","안국","종로3가","을지로3가","충무로","동대입구","약수","금호","옥수","압구정","신사","잠원","고속터미널","교대","남부터미널","양재","매봉","도곡","대치","학여울","대청","일원","수서","가락시장","경찰병원","오금"
    ],
    #----------------------------------------------------
    # 4호선
    #----------------------------------------------------
    "4호선": [
        "진접","오남","별내별가람","당고개",
        "상계","노원","창동","쌍문","수유","미아","미아사거리","길음","성신여대입구","한성대입구","혜화","동대문","동대문역사문화공원","충무로","명동","회현","서울역","숙대입구","삼각지","신용산","이촌","동작","총신대입구(이수)","사당","남태령",
        "선바위","경마공원","대공원","과천","정부과천청사","인덕원","평촌","범계","금정",
        "산본","수리산","대야미","반월","상록수","한대앞","중앙","고잔","초지","안산","신길온천","정왕","오이도"
    ],
    #----------------------------------------------------
    # 5호선
    #----------------------------------------------------
    "5호선": [
        "방화","개화산","김포공항","송정","마곡","발산","우장산","화곡","까치산","신정","목동","오목교","양평","영등포구청","영등포시장","신길","여의도","여의나루","마포","공덕","애오개","충정로","서대문","광화문","종로3가","을지로4가","동대문역사문화공원","청구","신금호","행당","왕십리","마장","답십리","장한평","군자","아차산","광나루","천호","강동","길동","굽은다리","명일","고덕","상일동","강일","미사","하남풍산","하남시청","하남검단산",
        "강동","둔촌동","올림픽공원","방이","오금","개롱","거여","마천"
    ],
    #----------------------------------------------------
    # 6호선
    #----------------------------------------------------
    "6호선": [
        # 응암순환 (응암->역촌->불광->독바위->연신내->구산->응암)
        "응암","역촌","불광","독바위","연신내","구산","응암",
        # 나머지 본선
        "새절","증산","디지털미디어시티","월드컵경기장","마포구청","망원","합정",
        "상수","광흥창","대흥","공덕","효창공원앞","삼각지",
        "녹사평","이태원","한강진","버티고개","약수","청구","신당","동묘앞","창신","보문","안암","고려대","월곡","상월곡","돌곶이","석계","태릉입구","화랑대","봉화산","신내"
    ],
    #----------------------------------------------------
    # 7호선
    #----------------------------------------------------
    "7호선": [
        "장암","도봉산","수락산","마들","노원","중계","하계","공릉","태릉입구","먹골","중화","상봉","면목","사가정","용마산","중곡","군자","어린이대공원","건대입구","자양","청담","강남구청","학동","논현","반포","고속터미널","내방","총신대입구(이수)","남성","숭실대입구","상도","장승배기","신대방삼거리","보라매","신풍","대림","남구로","가산디지털단지","철산","광명사거리","천왕","온수","까치울","부천종합운동장","춘의","신중동","부천시청","상동","삼산체육관","굴포천","부평구청","산곡","석남"
    ],
    #----------------------------------------------------
    # 8호선
    #----------------------------------------------------
    "8호선": [
        "별내","다산","동구릉","구리","장자호수공원","암사역사공원",
        "암사","천호","강동구청","몽촌토성","잠실","석촌","송파","가락시장","문정","장지","복정","남위례",
        "산성","남한산성입구","단대오거리","신흥","수진","모란"
    ],
    #----------------------------------------------------
    # 9호선
    #----------------------------------------------------
    "9호선": [
        "개화","김포공항","공항시장","신방화","마곡나루","양천향교","가양","증미","등촌","염창","신목동","선유도","당산",
        "국회의사당","여의도","샛강","노량진","노들","흑석","동작","구반포","신반포","고속터미널","사평","신논현",
        "언주","선정릉","삼성중앙","봉은사","종합운동장","삼전","석촌고분","석촌","송파나루","한성백제","올림픽공원","둔촌오륜","중앙보훈병원"
    ],
    #----------------------------------------------------
    # 수인분당선
    #----------------------------------------------------
    "수인분당선": [
        "청량리","왕십리","서울숲","압구정로데오","강남구청","선정릉","선릉","한티","도곡","구룡","개포동","대모산입구","수서","복정","가천대","태평","모란","야탑","이매","서현","수내","정자","미금","오리","죽전","보정","구성","신갈","기흥","상갈","청명","영통","망포","매탄권선","수원시청","매교","수원",
        "고색","오목천","어천","야목","사리","한대앞","중앙","고잔","초지","안산","신길온천","정왕","오이도","달월","월곶","소래포구","인천논현","호구포","남동인더스파크","원인재","연수","송도","인하대","숭의","신포","인천"
    ],
    #----------------------------------------------------
    # 신분당선
    #----------------------------------------------------
    "신분당선": [
        "신사","논현","신논현","강남","양재","양재시민의숲","청계산입구","판교","정자","미금","동천","수지구청","성복","상현","광교중앙","광교"
    ],
    #----------------------------------------------------
    # 경의중앙선
    #----------------------------------------------------
    "경의중앙선": [
        "임진강","운천","문산","파주","월롱","금촌","금릉","운정","야당","탄현","일산","풍산","백마","곡산","대곡","능곡","행신","강매","한국항공대","수색","디지털미디어시티","가좌","신촌(경의중앙선)","서울역",
        "가좌","홍대입구","서강대","공덕","효창공원앞","용산","이촌","서빙고","한남","옥수","응봉","왕십리","청량리","회기","중랑","상봉","망우","양원","구리","도농","양정","덕소","도심","팔당","운길산","양수","신원","국수","아신","오빈","양평","원덕","용문","지평"
    ],
    #----------------------------------------------------
    # 경춘선
    #----------------------------------------------------
    "경춘선": [
        "청량리","회기","중랑","상봉","망우","신내","갈매","별내","퇴계원","사릉","금곡","평내호평","천마산","마석","대성리","청평","상천","가평","굴봉산","백양리","강촌","김유정","남춘천","춘천",
        "상봉","광운대"
    ],
    #----------------------------------------------------
    # 서해선
    #----------------------------------------------------
    "서해선": [
        "일산","풍산","백마","곡산","대곡","능곡","김포공항","원종","부천종합운동장","소사","소새울","시흥대야","신천","신현","시흥시청","시흥능곡","달미","선부","초지","시우","원시"
    ],
    #----------------------------------------------------
    # 공항철도
    #----------------------------------------------------
    "공항철도": [
        "서울역","공덕","홍대입구","디지털미디어시티","마곡나루","김포공항","계양","검암","청라국제도시","영종","운서","공항화물청사","인천공항1터미널","인천공항2터미널"
    ],
    #----------------------------------------------------
    # 우이신설선
    #----------------------------------------------------
    "우이신설선": [
        "북한산우이","솔밭공원","4.19민주묘지","가오리","화계","삼양","삼양사거리","솔샘","북한산보국문","정릉","성신여대입구","보문","신설동"
    ],
    #----------------------------------------------------
    # 신림선
    #----------------------------------------------------
    "신림선": [
        "샛강","대방","서울지방병무청","보라매","보라매공원","보라매병원","당곡","신림","서원","서울대벤처타운","관악산"
    ],
    #----------------------------------------------------
    # 김포골드라인
    #----------------------------------------------------
    "김포골드라인": [
        "김포공항","고촌","풍무","사우","걸포북변","운양","장기","마산","구래","양촌"
    ],
    #----------------------------------------------------
    # 인천1호선
    #----------------------------------------------------
    "인천1호선": [
        "계양","귤현","박촌","임학","계산","경인교대입구","작전","갈산","부평구청","부평시장","부평","동수","부평삼거리","간석오거리","인천시청","예술회관","인천터미널","문학경기장","선학","신연수","원인재","동춘","동막","캠퍼스타운","테크노파크","지식정보단지","인천대입구","센트럴파크","국제업무지구","송도달빛축제공원"
    ],
    #----------------------------------------------------
    # 인천2호선
    #----------------------------------------------------
    "인천2호선": [
        "검단오류","왕길","검단사거리","마전","완정","독정","검암","검바위","아시아드경기장","서구청","가정","가정중앙시장","석남","서부여성회관","인천가좌","가재울","주안국가산단","주안","시민공원","석바위시장","인천시청","석천사거리","모래내시장","만수","남동구청","인천대공원","운연"
    ],
    #----------------------------------------------------
    # GTX-A
    #----------------------------------------------------
    "GTX-A": [
        "운정중앙","킨텍스","대곡","연신내","서울역",
        "수서","성남","구성","동탄"
    ],
    #----------------------------------------------------
    # 의정부경전철
    #----------------------------------------------------
    "의정부경전철": [
        "발곡","회룡","범골","경전철의정부","의정부시청","흥선","의정부중앙","동오","새말","경기도청북부청사","효자","곤제","어룡","송산","탑석"
    ],
    #----------------------------------------------------
    # 용인경전철
    #----------------------------------------------------
    "용인경전철": [
        "기흥","강남대","지석","어정","동백","초당","삼가","시청/용인대","명지대","김량장","용인중앙시장(용인예술과학대)","고진","보평","둔전","전대/에버랜드"
    ],
    #----------------------------------------------------
    # 경강선
    #----------------------------------------------------
    "경강선": [
        "판교","성남","이매","삼동","경기광주","초월","곤지암","신둔도예촌","이천","부발","세종대왕릉","여주"
    ],
}

#################################################
# 2) 그래프(인접 리스트) 생성 + BFS 최단 경로 함수
#################################################
def build_subway_graph(subway_lines):
    """모든 노선 정보를 바탕으로 '역 -> {인접역: {노선들}}' 형태의 그래프를 생성하고, 특정 구간을 제거."""
    graph = {}
    # (A) 모든 역을 키로 추가
    for line_name, stations in subway_lines.items():
        for st in stations:
            if st not in graph:
                graph[st] = {}
    # (B) 인접역 연결 (양방향)
    for line_name, stations in subway_lines.items():
        for i in range(len(stations) - 1):
            s1 = stations[i]
            s2 = stations[i+1]
            if s2 not in graph[s1]:
                graph[s1][s2] = set()
            graph[s1][s2].add(line_name)
    
            if s1 not in graph[s2]:
                graph[s2][s1] = set()
            graph[s2][s1].add(line_name)
    
    # ──────────────────────────────────────────
    # (C) [기존] 6호선 응암순환 단방향 처리
    # ──────────────────────────────────────────
    loop_6 = [
        ("응암", "역촌"),
        ("역촌", "불광"),
        ("불광", "독바위"),
        ("독바위", "연신내"),
        ("연신내", "구산"),
        ("구산", "응암")
    ]
    for (forward_a, forward_b) in loop_6:
        if forward_b in graph and forward_a in graph[forward_b]:
            if "6호선" in graph[forward_b][forward_a]:
                graph[forward_b][forward_a].remove("6호선")
                if len(graph[forward_b][forward_a]) == 0:
                    del graph[forward_b][forward_a]
    
    # ──────────────────────────────────────────
    # (D) [기존] GTX-A 서울역↔수서 구간 미개통 처리
    # ──────────────────────────────────────────
    gtx_a_block_pairs = [("서울역", "수서")]
    for (stA, stB) in gtx_a_block_pairs:
        if stA in graph and stB in graph[stA]:
            if "GTX-A" in graph[stA][stB]:
                graph[stA][stB].remove("GTX-A")
                if len(graph[stA][stB]) == 0:
                    del graph[stA][stB]
        if stB in graph and stA in graph[stB]:
            if "GTX-A" in graph[stB][stA]:
                graph[stB][stA].remove("GTX-A")
                if len(graph[stB][stA]) == 0:
                    del graph[stB][stA]
    
    # ──────────────────────────────────────────
    # (E) [추가] 특정 구간 제거 (예: "구산" ↔ "연신내", "구로" ↔ "인천")
    # ──────────────────────────────────────────
    removal_list = [
        ("6호선", "구산", "연신내", "forward"),
        ("6호선", "응암", "구산", "forward"),
        ("6호선", "역촌", "응암", "forward"),
        ("6호선", "불광", "역촌", "forward"),
        ("6호선", "독바위", "불광", "forward"),
        ("6호선", "연신내", "독바위", "forward"),
        ("1호선", "구로", "인천", "both"),
        ("1호선", "신창", "금천구청", "both"),
        ("1호선", "광명", "병점", "both"),
        ("2호선", "충정로", "성수", "both"),
        ("2호선", "신설동", "신도림", "both"),
        ("5호선", "하남검단산", "강동", "both"),
        ("1호선", "연천", "구일", "both"),
        ("1호선", "연천", "구로", "both"),
        ("1호선", "연천", "금천구청", "both"),
        ("1호선", "연천", "병점", "both"),
        ("1호선", "구일", "구로", "both"),
        ("1호선", "구일", "금천구청", "both"),
        ("1호선", "구일", "병점", "both"),
        ("1호선", "구로", "금천구청", "both"),
        ("1호선", "구로", "병점", "both"),
        ("1호선", "금천구청", "병점", "both"),
    ]
    for (line, sta_a, sta_b, direction) in removal_list:
        remove_line_connection(graph, line, sta_a, sta_b, direction)
    
    return graph

def bfs_shortest_path(graph, start_station, end_station):
    if start_station not in graph or end_station not in graph:
        return []
    visited = set()
    queue = deque([[start_station]])
    solutions = []
    found_distance = None

    while queue:
        path = queue.popleft()
        cur = path[-1]

        if found_distance is not None and len(path) - 1 > found_distance:
            continue

        if cur == end_station:
            distance_now = len(path) - 1
            if found_distance is None:
                found_distance = distance_now
            if distance_now == found_distance:
                solutions.append(path)
            continue

        if cur not in visited:
            visited.add(cur)
            for neighbor in graph[cur]:
                new_path = path + [neighbor]
                queue.append(new_path)

    if not solutions:
        return []

    min_transfer = None
    best_path = None
    for candidate in solutions:
        analysis = analyze_route(graph, candidate)
        tcount = analysis["transfer_count"]
        if min_transfer is None or tcount < min_transfer:
            min_transfer = tcount
            best_path = candidate

    return best_path

###########################################################
# 3) 경로(역 리스트)에서 환승역·환승노선 등을 분석하는 함수
###########################################################
def analyze_route(graph, path):
    if len(path) < 2:
        return {
            "transfer_count": 0,
            "transfer_stations": [],
            "route_info": [(st, []) for st in path]
        }

    transfer_count = 0
    transfer_stations = []
    route_info = []

    st_first = path[0]
    st_second = path[1]
    if st_second not in graph[st_first]:
        return {
            "transfer_count": 99999999,
            "transfer_stations": [],
            "route_info": []
        }
    current_lines = graph[st_first][st_second]
    route_info.append((st_first, list(current_lines)))

    for i in range(len(path) - 1):
        st1 = path[i]
        st2 = path[i+1]
        if st2 not in graph[st1]:
            return {
                "transfer_count": 99999999,
                "transfer_stations": [],
                "route_info": []
            }
        lines_between = graph[st1][st2]
        common = current_lines.intersection(lines_between)
        if not common:
            transfer_count += 1
            transfer_stations.append(st1)
            current_lines = lines_between
        route_info.append((st2, list(current_lines)))

    return {
        "transfer_count": transfer_count,
        "transfer_stations": transfer_stations,
        "route_info": route_info
    }

###########################################################
# 4) “기후동행카드 범위 역 정보” 엑셀 => dict 로딩
###########################################################
def load_climate_pass_info(excel_path):
    df = pd.read_excel(excel_path)
    info_dict = {}
    for idx, row in df.iterrows():
        station = str(row["역명"]).strip()
        line = str(row["노선"]).strip()
        board = str(row["승차"]).strip()
        alight = str(row["하차"]).strip()
        info_dict[(station, line)] = (board, alight)
    return info_dict

def is_fully_in_climate_pass(path_analysis, climate_pass_info):
    for (station, lines_used) in path_analysis["route_info"]:
        if not lines_used:
            return False
        any_line_ok = False
        for line_name in lines_used:
            key = (station, line_name)
            if key in climate_pass_info:
                board, alight = climate_pass_info[key]
                if board == "O" and alight == "O":
                    any_line_ok = True
                    break
        if not any_line_ok:
            return False

    return True

###########################################################
# 5) 기타 보조 함수들
###########################################################
def attempt_direct_route_same_line(subway_lines, line_name, start_st, end_st):
    if line_name not in subway_lines:
        return None
    stations = subway_lines[line_name]
    if start_st not in stations or end_st not in stations:
        return None

    idx_s = stations.index(start_st)
    idx_e = stations.index(end_st)
    if idx_s < idx_e:
        return stations[idx_s:idx_e+1]
    elif idx_s > idx_e:
        return stations[idx_e:idx_s+1][::-1]
    else:
        return [start_st]

def remove_line_connection(graph, line_name, station_a, station_b, direction="both"):
    if direction == "both":
        if station_a in graph and station_b in graph[station_a]:
            if line_name in graph[station_a][station_b]:
                graph[station_a][station_b].remove(line_name)
                if not graph[station_a][station_b]:
                    del graph[station_a][station_b]
        if station_b in graph and station_a in graph[station_b]:
            if line_name in graph[station_b][station_a]:
                graph[station_b][station_a].remove(line_name)
                if not graph[station_b][station_a]:
                    del graph[station_b][station_a]
    
    elif direction == "forward":
        if station_a in graph and station_b in graph[station_a]:
            if line_name in graph[station_a][station_b]:
                graph[station_a][station_b].remove(line_name)
                if not graph[station_a][station_b]:
                    del graph[station_a][station_b]
    
    elif direction == "reverse":
        if station_b in graph and station_a in graph[station_b]:
            if line_name in graph[station_b][station_a]:
                graph[station_b][station_a].remove(line_name)
                if not graph[station_b][station_a]:
                    del graph[station_b][station_a]

def remove_all_gtx_a_edges(graph_origin):
    stations = list(graph_origin.keys())
    for sA in stations:
        neighbors = list(graph_origin[sA].keys())
        for sB in neighbors:
            lines = graph_origin[sA][sB]
            if "GTX-A" in lines:
                lines.remove("GTX-A")
                if not lines:
                    del graph_origin[sA][sB]
    return graph_origin

###########################################################
# [추가] 개통 전 구간 차단을 위한 함수들
###########################################################
def block_segment_in_graph(graph, line_name, start, end):
    """
    subway_lines에 정의된 해당 노선의 구간에서, start ~ end 사이의
    인접 간선들을 모두 제거(양방향)한다.
    """
    stations = subway_lines.get(line_name)
    if not stations:
        return
    try:
        idx_start = stations.index(start)
        idx_end = stations.index(end)
    except ValueError:
        return
    if idx_start <= idx_end:
        segment_stations = stations[idx_start:idx_end+1]
    else:
        segment_stations = stations[idx_end:idx_start+1][::-1]
    for i in range(len(segment_stations) - 1):
        sA = segment_stations[i]
        sB = segment_stations[i+1]
        remove_line_connection(graph, line_name, sA, sB, direction="both")

def adjust_graph_for_openings(graph, file_date):
    """
    file_date: 정수형 YYYYMMDD
    <수도권 지하철 개통 정리>에 따라 개통일 이전인 구간(양방향)을 차단.
    
    개통 정리:
      - 23년07월01일 서해선 대곡~소사 개통  
      - 23년08월26일 서해선 일산~대곡 개통  
      - 23년12월16일 1호선 연천~소요산 개통  
      - 24년03월30일 GTX-A 수서~동탄 개통  
      - 24년03월30일 경강선 성남 개통  
      - 24년06월29일 GTX-A 구성 개통  
      - 24년08월10일 8호선 별내~암사 개통  
      - 24년12월28일 GTX-A 운정중앙~서울 개통
    """
    conditions = [
        {"line": "서해선", "start": "대곡", "end": "소사", "open_date": 20230701},
        {"line": "서해선", "start": "일산", "end": "대곡", "open_date": 20230826},
        {"line": "1호선", "start": "연천", "end": "소요산", "open_date": 20231216},
        # GTX-A 수서~동탄: 세분화 — (수서~성남)와 (구성~동탄)는 24년03월30일, 단 (성남~구성)는 별도
        {"line": "GTX-A", "start": "수서", "end": "성남", "open_date": 20240330},
        {"line": "GTX-A", "start": "구성", "end": "동탄", "open_date": 20240330},
        {"line": "GTX-A", "start": "성남", "end": "구성", "open_date": 20240629},
        # 경강선 성남 개통: 해당 노선에서 성남이 포함된 구간 양쪽 (판교~성남, 성남~이매) 차단
        {"line": "경강선", "start": "판교", "end": "성남", "open_date": 20240330},
        {"line": "경강선", "start": "성남", "end": "이매", "open_date": 20240330},
        {"line": "8호선", "start": "별내", "end": "암사", "open_date": 20240810},
        {"line": "GTX-A", "start": "운정중앙", "end": "서울역", "open_date": 20241228},
    ]
    for cond in conditions:
        if file_date < cond["open_date"]:
            block_segment_in_graph(graph, cond["line"], cond["start"], cond["end"])
    return graph

###########################################################
# 6) 메인 로직
###########################################################
def main():
    # (A) 기후동행카드 범위 엑셀 로딩
    climate_pass_info = load_climate_pass_info("기후동행카드 지원범위.xlsx")
    
    # (B) 지정된 9개 CSV 파일이 위치한 폴더 내 모든 파일 읽기
    data_dir = r"C:\Users\bag43\Desktop\자료실\자료실\외부 활동\▲동아리\DSL\2025-1\EDA\기동카 월별정리 데이터셋"
    csv_files = glob.glob(os.path.join(data_dir, "*.csv"))
    csv_files.sort()  # 정렬하여 순서대로 처리
    
    # (C) 상위 40%와 상위 100% 두 조건
    thresholds = [0.4, 1.0]
    
    # 각 CSV 파일마다 처리 (총 9개 파일 → 각 파일당 2번씩, 총 18회)
    for file_idx, csv_file in enumerate(csv_files):
        # 파일명에서 8자리 날짜 추출 (예: 20230331)
        base_name = os.path.basename(csv_file)
        m = re.search(r'(\d{8})', base_name)
        if m:
            file_date_str = m.group(1)
            file_date = int(file_date_str)
        else:
            print(f"[경고] {csv_file}에서 날짜 추출 실패")
            continue
        
        # (D) 기본 그래프 생성 후, 해당 파일 날짜 기준 개통 전 구간 차단
        base_graph = build_subway_graph(subway_lines)
        graph = adjust_graph_for_openings(copy.deepcopy(base_graph), file_date)
        
        # (E) 해당 CSV 파일의 메인 데이터 로딩
        df_main = pd.read_csv(csv_file)
        
        segment_passenger = defaultdict(int)
        route_records = []
        
        for idx, row in df_main.iterrows():
            start_line = row["승차_호선"].strip()
            start_st = row["승차_역"].strip()
            end_line = row["하차_호선"].strip()
            end_st = row["하차_역"].strip()
            passenger = row["일반_승객수"]
    
            # 1) 동일 노선이면 직통 시도
            direct_path = None
            if start_line == end_line:
                direct_path = attempt_direct_route_same_line(subway_lines, start_line, start_st, end_st)
                if direct_path:
                    valid = True
                    for i in range(len(direct_path) - 1):
                        sA = direct_path[i]
                        sB = direct_path[i+1]
                        if sB not in graph.get(sA, {}):
                            valid = False
                            break
                    if not valid:
                        direct_path = None
    
            if direct_path and len(direct_path) > 1:
                path = direct_path
            else:
                path_with_gtx = bfs_shortest_path(graph, start_st, end_st)
                if not path_with_gtx:
                    continue
                analysis_with_gtx = analyze_route(graph, path_with_gtx)
                used_gtx = any(("GTX-A" in lines for _, lines in analysis_with_gtx["route_info"]))
                if used_gtx:
                    graph_no_gtx = remove_all_gtx_a_edges(copy.deepcopy(graph))
                    path_no_gtx = bfs_shortest_path(graph_no_gtx, start_st, end_st)
                    if path_no_gtx:
                        len_with = len(path_with_gtx)
                        len_no = len(path_no_gtx)
                        if len_with <= 0.67 * len_no:
                            path = path_with_gtx
                        else:
                            path = path_no_gtx
                    else:
                        path = path_with_gtx
                else:
                    path = path_with_gtx
    
            analysis = analyze_route(graph, path)
            climate_ok = is_fully_in_climate_pass(analysis, climate_pass_info)
    
            # 구간별 누적 승객수 집계
            for i in range(len(path) - 1):
                sA = path[i]
                sB = path[i+1]
                seg = tuple(sorted((sA, sB)))
                segment_passenger[seg] += passenger
    
            rec = {
                "start_station": start_st,
                "start_line": start_line,
                "end_station": end_st,
                "end_line": end_line,
                "transfer_count": analysis["transfer_count"],
                "transfer_stations": ";".join(analysis["transfer_stations"]),
                "cumulative_passenger": passenger,
                "climate_pass": "O" if climate_ok else "X"
            }
            route_records.append(rec)
    
        # 구간별 누적 승객수 내림차순 정렬
        seg_sorted = sorted(segment_passenger.items(), key=lambda x: x[1], reverse=True)
        if not seg_sorted:
            print(f"[오류] {csv_file}에 유효한 구간 데이터가 없습니다.")
            continue
    
        # (F) 상위 조건(40%와 100%) 각각에 대해 엑셀 저장 (총 2회)
        for thresh in thresholds:
            top_count = max(1, math.ceil(len(seg_sorted) * thresh))
            top_segments = seg_sorted[:top_count]
    
            rows_for_excel = []
            for seg, val in top_segments:
                stationA, stationB = seg
                lines = graph.get(stationA, {}).get(stationB, set())
                is_segment_climate_ok = False
                for line_candidate in lines:
                    keyA = (stationA, line_candidate)
                    keyB = (stationB, line_candidate)
                    if keyA in climate_pass_info and keyB in climate_pass_info:
                        boardA, alightA = climate_pass_info[keyA]
                        boardB, alightB = climate_pass_info[keyB]
                        if boardA == "O" and alightA == "O" and boardB == "O" and alightB == "O":
                            is_segment_climate_ok = True
                            break
                climate_ok_str = "O" if is_segment_climate_ok else "X"
                row_dict = {
                    "start_station": stationA,
                    "end_station": stationB,
                    "lines": ",".join(lines) if lines else "",
                    "cumulative_passenger": val,
                    "transfer_info": "(단일 구간은 환승 없음)",
                    "climate_pass": climate_ok_str
                }
                rows_for_excel.append(row_dict)
    
            df_top = pd.DataFrame(rows_for_excel)
            thresh_percent = int(thresh * 100)
            output_filename = f"[0202최종]상위{thresh_percent:02d}퍼센트_구간정보_{file_idx}번째_{file_date_str}.xlsx"
            df_top.to_excel(output_filename, index=False)
            print(f"저장 완료: {output_filename}")

if __name__ == "__main__":
    main()


저장 완료: [0202최종]상위40퍼센트_구간정보_0번째_20230331.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_0번째_20230331.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_1번째_20230430.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_1번째_20230430.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_2번째_20230531.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_2번째_20230531.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_3번째_20230630.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_3번째_20230630.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_4번째_20230731.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_4번째_20230731.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_5번째_20230831.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_5번째_20230831.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_6번째_20230930.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_6번째_20230930.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_7번째_20231031.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_7번째_20231031.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_8번째_20231131.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_8번째_20231131.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_9번째_20231231.xlsx
저장 완료: [0202최종]상위100퍼센트_구간정보_9번째_20231231.xlsx
저장 완료: [0202최종]상위40퍼센트_구간정보_10번째_20240131.xlsx
저장 완료: [0202최종]상위100퍼센트